<a href="https://colab.research.google.com/github/Alanazi433/GenAI/blob/main/GenAI/HW7/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install mido music21


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.7 MB/s eta 0:00:00


In [1]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()

# This will prompt you to upload a file from your computer.


Saving bach_cello_suites.zip to bach_cello_suites.zip


In [2]:
import zipfile
import os

# Specify the uploaded file path
zip_path = '/content/bach_cello_suites.zip'

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/bach_cello_suites/')

# Verify the extraction
midi_folder = '/content/bach_cello_suites/'
print("Files extracted to:", midi_folder)
print("MIDI files:", os.listdir(midi_folder))


Files extracted to: /content/bach_cello_suites/
MIDI files: ['data']


In [3]:
import os

# Check the contents of the "data" folder
nested_folder = '/content/bach_cello_suites/data'
print("Files in 'data' folder:", os.listdir(nested_folder))


Files in 'data' folder: ['cs4-3cou.mid', 'cs4-1pre.mid', 'cs3-4sar.mid', 'cs3-3cou.mid', 'cs6-5gav.mid', 'cs2-6gig.mid', 'cs1-3cou.mid', 'cs4-4sar.mid', 'cs1-5men.mid', 'cs2-2all.mid', 'cs2-5men.mid', 'cs2-4sar.mid', 'cs6-2all.mid', 'cs6-3cou.mid', 'cs1-6gig.mid', 'cs5-5gav.mid', 'cs5-4sar.mid', 'cs3-1pre.mid', 'cs3-2all.mid', 'durations', 'cs5-2all.mid', 'cs1-4sar.mid', 'cs2-1pre.mid', 'cs5-1pre.mid', 'cs6-4sar.mid', 'cs2-3cou.mid', 'cs1-1pre.mid', 'cs6-6gig.mid', 'cs4-6gig.mid', 'cs4-2all.mid', 'cs3-6gig.mid', 'cs6-1pre.mid', 'cs3-5bou.mid', 'cs4-5bou.mid', 'cs5-3cou.mid', 'cs1-2all.mid', 'cs5-6gig.mid', 'notes']


In [4]:
midi_folder = '/content/bach_cello_suites/data'  # Updated to point to the correct folder

# List all MIDI files
midi_files = [os.path.join(midi_folder, f) for f in os.listdir(midi_folder) if f.endswith('.mid')]
print(f"Found {len(midi_files)} MIDI files.")


Found 36 MIDI files.


In [5]:
from music21 import converter, instrument, note, chord

def parse_midi(file_path):
    """
    Extract notes and chords from a MIDI file.
    :param file_path: Path to the MIDI file.
    :return: List of note and chord representations.
    """
    try:
        midi = converter.parse(file_path)
        notes = []

        # Check for instrument parts or flat structure
        parts = instrument.partitionByInstrument(midi)
        if parts:  # If multiple instruments
            for part in parts.parts:  # Iterate through all parts
                for element in part.recurse():
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))
        else:  # Single flat structure
            for element in midi.flat.notes:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        return notes

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return []

# Parse the MIDI files again
all_notes = []
for midi_file in midi_files:
    notes = parse_midi(midi_file)
    all_notes.append(notes)

print(f"Reprocessed {len(midi_files)} MIDI files.")
print(f"Example notes from the first file: {all_notes[0][:10]}")


Reprocessed 36 MIDI files.
Example notes from the first file: ['E-3', 'E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-3', 'E-2', 'D3']


In [6]:
# Identify files with no notes
empty_files = [midi_files[i] for i, notes in enumerate(all_notes) if len(notes) == 0]
print(f"Files with no notes: {empty_files}")

# Keep only files with valid notes
valid_notes = [notes for notes in all_notes if len(notes) > 0]
print(f"Number of valid MIDI files: {len(valid_notes)}")


Files with no notes: []
Number of valid MIDI files: 36


In [7]:
import pickle

# Save the valid notes to a file
output_path = '/content/valid_parsed_notes.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(valid_notes, f)

print(f"Valid parsed notes saved to: {output_path}")


Valid parsed notes saved to: /content/valid_parsed_notes.pkl


In [8]:
import pickle

# Load parsed notes from the previous step
with open('/content/valid_parsed_notes.pkl', 'rb') as f:
    valid_notes = pickle.load(f)

print(f"Loaded {len(valid_notes)} files of parsed notes.")
print(f"Example notes from the first file: {valid_notes[0][:10]}")


Loaded 36 files of parsed notes.
Example notes from the first file: ['E-3', 'E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-3', 'E-2', 'D3']


In [9]:
from collections import Counter

# Flatten the list of all notes to create a single sequence
all_notes_flat = [note for file_notes in valid_notes for note in file_notes]

# Count the frequency of each note
note_counter = Counter(all_notes_flat)

# Create a vocabulary (mapping of notes to tokens)
vocab = {note: i for i, note in enumerate(note_counter.keys())}

# Add reverse mapping for decoding (optional, useful for generation)
reverse_vocab = {i: note for note, i in vocab.items()}

print(f"Vocabulary size: {len(vocab)}")
print("Sample vocab:", list(vocab.items()))


Vocabulary size: 127
Sample vocab: [('E-3', 0), ('B-2', 1), ('C3', 2), ('G#2', 3), ('F2', 4), ('D3', 5), ('E-2', 6), ('F3', 7), ('G3', 8), ('A3', 9), ('B-3', 10), ('G2', 11), ('G#3', 12), ('C4', 13), ('D2', 14), ('D4', 15), ('E-4', 16), ('F4', 17), ('G4', 18), ('A2', 19), ('B2', 20), ('C2', 21), ('B3', 22), ('E3', 23), ('C#4', 24), ('F#3', 25), ('C#3', 26), ('E4', 27), ('E2', 28), ('F#2', 29), ('C#2', 30), ('11.0', 31), ('7.9', 32), ('2.4', 33), ('F#4', 34), ('A4', 35), ('B4', 36), ('4.6', 37), ('8.9', 38), ('0.2', 39), ('5.7', 40), ('4.7', 41), ('1.2.4', 42), ('11.1', 43), ('9.11.1', 44), ('6.7', 45), ('2.4.6', 46), ('4.6.7', 47), ('2.6', 48), ('D5', 49), ('4.8', 50), ('11.2', 51), ('10.11.1.2', 52), ('C#5', 53), ('11.4', 54), ('11.1.2', 55), ('9.11', 56), ('9', 57), ('G#4', 58), ('6.8', 59), ('6.9', 60), ('1.2', 61), ('6.11', 62), ('9.2', 63), ('1.4', 64), ('9.1', 65), ('9.11.1.2', 66), ('7.11', 67), ('0.2.4', 68), ('6', 69), ('5.8', 70), ('8.11', 71), ('2.5', 72), ('2.4.6.7', 73), (

In [10]:
# Convert notes to tokens
tokenized_notes = [[vocab[note] for note in file_notes] for file_notes in valid_notes]

print(f"Tokenized {len(tokenized_notes)} files of notes.")
print(f"Example tokenized notes from the first file: {tokenized_notes[0]}")


Tokenized 36 files of notes.
Example tokenized notes from the first file: [0, 0, 1, 2, 3, 4, 5, 0, 6, 5, 0, 7, 8, 7, 8, 9, 10, 7, 8, 0, 2, 9, 10, 1, 3, 11, 0, 3, 1, 2, 2, 5, 0, 7, 8, 12, 11, 3, 1, 1, 2, 5, 0, 7, 8, 7, 8, 12, 8, 7, 0, 1, 0, 5, 0, 6, 1, 2, 5, 0, 5, 0, 7, 8, 0, 8, 5, 2, 10, 9, 6, 2, 5, 0, 7, 0, 7, 8, 9, 7, 9, 0, 5, 13, 10, 14, 5, 0, 7, 8, 7, 8, 9, 10, 8, 10, 7, 0, 15, 13, 9, 13, 8, 7, 16, 15, 10, 15, 9, 8, 17, 16, 13, 16, 9, 7, 16, 15, 10, 15, 8, 0, 13, 9, 7, 7, 8, 9, 10, 13, 13, 0, 9, 10, 13, 15, 16, 16, 9, 2, 13, 15, 16, 17, 18, 18, 9, 17, 16, 15, 13, 15, 10, 0, 13, 7, 9, 10, 1, 10, 9, 8, 7, 8, 7, 0, 5, 1, 2, 5, 0, 7, 7, 8, 9, 10, 5, 2, 4, 1, 1, 0, 0, 1, 2, 3, 4, 5, 0, 6, 5, 0, 7, 8, 7, 8, 9, 10, 7, 8, 0, 2, 9, 10, 1, 3, 11, 0, 3, 1, 2, 2, 5, 0, 7, 8, 12, 11, 3, 1, 1, 2, 5, 0, 7, 8, 7, 8, 12, 8, 7, 0, 1, 0, 5, 0, 6, 1, 2, 5, 0, 5, 0, 7, 8, 0, 8, 5, 2, 10, 9, 6, 2, 5, 0, 7, 0, 7, 8, 9, 7, 9, 0, 5, 13, 10, 14, 5, 0, 7, 8, 7, 8, 9, 10, 8, 10, 7, 0, 15, 13, 9, 13, 8, 7, 16,

In [11]:
# Save tokenized notes
with open('/content/tokenized_notes.pkl', 'wb') as f:
    pickle.dump(tokenized_notes, f)

# Save vocabulary
with open('/content/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

print("Tokenized notes and vocabulary saved.")


Tokenized notes and vocabulary saved.


In [12]:
import numpy as np

def create_training_sequences(tokenized_notes, seq_length=50):
    """
    Creates input-output pairs for training the Transformer.
    :param tokenized_notes: List of tokenized notes (one list per file).
    :param seq_length: Length of input sequences.
    :return: input_sequences, output_tokens
    """
    input_sequences = []
    output_tokens = []

    for file_tokens in tokenized_notes:
        for i in range(len(file_tokens) - seq_length):
            input_seq = file_tokens[i:i + seq_length]
            output_token = file_tokens[i + seq_length]

            input_sequences.append(input_seq)
            output_tokens.append(output_token)

    return np.array(input_sequences), np.array(output_tokens)

# Define sequence length
sequence_length = 50

# Create training sequences
input_sequences, output_tokens = create_training_sequences(tokenized_notes, seq_length=sequence_length)

print(f"Input sequences shape: {input_sequences.shape}")
print(f"Output tokens shape: {output_tokens.shape}")


Input sequences shape: (27502, 50)
Output tokens shape: (27502,)


In [13]:
# Save training data
np.save('/content/input_sequences.npy', input_sequences)
np.save('/content/output_tokens.npy', output_tokens)

print("Training data saved as numpy arrays.")


Training data saved as numpy arrays.


In [14]:
import tensorflow as tf
from tensorflow.keras import layers
vocab_size = len(vocab)
seq_length=50
# Updated Positional Encoding layer
class PositionalEncoding(layers.Layer):
    def __init__(self, seq_length, d_model):
        """
        Initialize PositionalEncoding.
        :param seq_length: Maximum sequence length.
        :param d_model: Dimensionality of the embedding space.
        """
        super(PositionalEncoding, self).__init__()
        self.seq_length = seq_length
        self.d_model = d_model

        # Precompute the positional encodings
        self.positional_encodings = self.compute_positional_encodings()

    def compute_positional_encodings(self):
        """
        Precompute positional encodings for the given sequence length and embedding size.
        """
        positions = tf.range(self.seq_length, dtype=tf.float32)[:, tf.newaxis]
        dimensions = tf.range(self.d_model, dtype=tf.float32)[tf.newaxis, :]

        angle_rates = 1 / tf.pow(10000, (2 * (dimensions // 2)) / tf.cast(self.d_model, tf.float32))
        angle_rads = positions * angle_rates

        # Apply sin to even indices and cos to odd indices
        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])

        # Combine sines and cosines
        positional_encodings = tf.concat([sines, cosines], axis=-1)
        return positional_encodings[tf.newaxis, ...]

    def call(self, inputs):
        """
        Add positional encodings to the input embeddings.
        :param inputs: Embedding inputs of shape (batch_size, seq_length, d_model).
        :return: Input embeddings with positional encodings added.
        """
        batch_size = tf.shape(inputs)[0]
        return inputs + self.positional_encodings[:, :tf.shape(inputs)[1], :]

# Define the Transformer model
def create_transformer_model(vocab_size, seq_length, d_model=128, num_heads=4, num_layers=4, dropout_rate=0.3):
    inputs = layers.Input(shape=(seq_length,), dtype=tf.int32)
    x = layers.Embedding(vocab_size, d_model)(inputs)
    x = PositionalEncoding(seq_length, d_model)(x)

    # Transformer layers
    for _ in range(num_layers):
        attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attention)
        feedforward = layers.Dense(d_model, activation='relu')(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + feedforward)
        x = layers.Dropout(dropout_rate)(x)

    # Predict a token for each position
    outputs = layers.Dense(vocab_size, activation='softmax')(x)

    # Compile the model
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_transformer_model(vocab_size, seq_length)




In [15]:
def create_training_sequences_for_seq2seq(tokenized_notes, seq_length=50):
    input_sequences = []
    output_sequences = []
    for file_tokens in tokenized_notes:
        for i in range(len(file_tokens) - seq_length):
            input_seq = file_tokens[i:i + seq_length]
            output_seq = file_tokens[i + 1:i + 1 + seq_length]

            input_sequences.append(input_seq)
            output_sequences.append(output_seq)

    return np.array(input_sequences), np.array(output_sequences)

input_sequences, output_sequences = create_training_sequences_for_seq2seq(tokenized_notes, seq_length=50)


In [16]:

batch_size = 64



In [17]:
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, output_sequences))
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

history = model.fit(dataset, epochs=50)


Epoch 1/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.0776 - loss: 3.5260
Epoch 2/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0882 - loss: 3.3499
Epoch 3/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.0887 - loss: 3.3484
Epoch 4/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0885 - loss: 3.3462
Epoch 5/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0878 - loss: 3.4725
Epoch 6/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.0886 - loss: 3.3484
Epoch 7/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.0894 - loss: 3.3441
Epoch 8/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0889 - loss: 3.3410
Epoch 9/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.0894 - loss: 3.3413
Epoch 10/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.0892 - loss: 3.3433
Epoch 11/50
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.0893 - loss: 3.3411
Epoch 12/50
429/429 ━━━━━━━━━━━━━━

In [18]:
# Save the entire model as a single file
model.save('transformer_model.h5')  # This saves everything: weights + architecture
# Save the model weights only
model.save_weights('transformer.weights.h5')

# Save the model architecture
with open('transformer_architecture.json', 'w') as f:
    f.write(model.to_json())


In [19]:
from google.colab import files

# Download the saved files
files.download('transformer_model.h5')  # For the entire model
files.download('transformer.weights.h5')  # For weights only
files.download('transformer_architecture.json')  # For architecture only


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# Add the sample_next_token function above generate_music
def sample_next_token(probabilities, temperature=1.0):
    probabilities = np.log(probabilities + 1e-10) / temperature
    probabilities = np.exp(probabilities)
    probabilities /= np.sum(probabilities)
    return np.random.choice(len(probabilities), p=probabilities)

# Existing generate_music function
def generate_music(model, seed_sequence, vocab, reverse_vocab, seq_length=50, num_tokens=100, temperature=1.0):
    generated = list(seed_sequence)
    print(f"Initial seed sequence: {generated[:10]}")  # Debug: Initial sequence

    for i in range(num_tokens):
        try:
            input_seq = np.array(generated[-seq_length:]).reshape(1, seq_length)
            print(f"\nStep {i+1}/{num_tokens}: Input sequence shape: {input_seq.shape}")

            predictions = model.predict(input_seq, verbose=0)
            print(f"Predictions shape: {predictions.shape}")

            predictions = predictions[0, -1, :]
            print(f"Predictions shape after extracting probabilities: {predictions.shape}")

            next_token = sample_next_token(predictions, temperature)
            print(f"Sampled token: {next_token}")

            generated.append(next_token)
        except Exception as e:
            print(f"Error in step {i+1}: {e}")
            break

    try:
        generated_notes_and_durations = [reverse_vocab[token] for token in generated]
        print(f"Generated notes and durations: {generated_notes_and_durations[:10]}")
        return generated_notes_and_durations
    except Exception as e:
        print(f"Error during decoding: {e}")
        return []


In [21]:
# Use the first sequence in the dataset as the seed
seed_sequence = input_sequences[0]

# Generate notes and durations
generated_notes_and_durations = generate_music(model, seed_sequence, vocab, reverse_vocab, num_tokens=100)

# Print the first 10 generated (note, duration) pairs
print(f"Generated notes and durations: {generated_notes_and_durations[:10]}")


Initial seed sequence: [0, 0, 1, 2, 3, 4, 5, 0, 6, 5]

Step 1/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 8

Step 2/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 25

Step 3/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 0

Step 4/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 17

Step 5/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 22

Step 6/100: Input sequence shape: (1, 50)
Predictions shape: (1, 50, 127)
Predictions shape after extracting probabilities: (127,)
Sampled token: 34

Step 7/100: Input sequence shape: (1, 50)
Predi

In [22]:
print(f"Vocabulary keys (sample): {list(vocab.keys())[:10]}")
print(f"Reverse vocabulary values (sample): {list(reverse_vocab.values())[:10]}")

# Check if specific notes exist
notes_to_check = ['A3', 'D3', 'B-3', 'C#3', 'G3']
missing_notes = [note for note in notes_to_check if note not in vocab]
print(f"Missing notes in vocab: {missing_notes}")


Vocabulary keys (sample): ['E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-2', 'F3', 'G3', 'A3']
Reverse vocabulary values (sample): ['E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-2', 'F3', 'G3', 'A3']
Missing notes in vocab: []


In [23]:
# Normalize reverse_vocab keys
reverse_vocab = {i: str(note).strip() for i, note in reverse_vocab.items()}

# Normalize generated_notes
generated_notes_and_durations = [str(token).strip() for token in generated_notes_and_durations]

# Debugging output
print(f"Normalized keys in reverse_vocab: {list(reverse_vocab.values())[:10]}")
print(f"Normalized tokens in generated_notes: {generated_notes_and_durations[:10]}")


Normalized keys in reverse_vocab: ['E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-2', 'F3', 'G3', 'A3']
Normalized tokens in generated_notes: ['E-3', 'E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-3', 'E-2', 'D3']


In [24]:
# Check for missing tokens
missing_tokens = [token for token in generated_notes_and_durations if token not in reverse_vocab.values()]
if missing_tokens:
    print(f"Missing tokens in reverse_vocab after normalization: {missing_tokens}")
else:
    print("All tokens in generated_notes match reverse_vocab keys.")


All tokens in generated_notes match reverse_vocab keys.


In [25]:
# Debug tokens that are missing
for token in generated_notes_and_durations:
    if token not in reverse_vocab.values():
        print(f"Token not in reverse_vocab: '{token}' (Type: {type(token)})")


In [26]:
# Rebuild reverse_vocab from vocab
reverse_vocab = {i: str(note).strip() for note, i in vocab.items()}


In [27]:
# Map generated tokens to notes or chords
mapped_notes = []
for token in generated_notes_and_durations:
    if token in reverse_vocab.values():
        mapped_notes.append(token)
    else:
        raise ValueError(f"Token {token} not found in reverse_vocab. This should not happen after normalization.")

print(f"Validated mapped notes: {mapped_notes[:10]}")


Validated mapped notes: ['E-3', 'E-3', 'B-2', 'C3', 'G#2', 'F2', 'D3', 'E-3', 'E-2', 'D3']


In [28]:
from music21 import stream, note, chord

def notes_to_midi(mapped_notes, output_file='generated_music_fixed.mid'):
    """
    Convert mapped notes (chords or single notes) into a MIDI file.
    :param mapped_notes: List of mapped notes/chords.
    :param output_file: Name of the output MIDI file.
    """
    midi_stream = stream.Stream()

    for element in mapped_notes:
        if '.' in element:  # Chord
            notes_in_chord = element.split('.')  # Assume elements are note names like "A3.C4.E4"
            midi_stream.append(chord.Chord(notes_in_chord))
        else:  # Single note
            midi_stream.append(note.Note(element))  # Directly pass the note name

    # Save the MIDI file
    midi_stream.write('midi', fp=output_file)
    print(f"MIDI file saved as {output_file}")

# Convert to MIDI
notes_to_midi(mapped_notes)


MIDI file saved as generated_music_fixed.mid


In [29]:
# Convert and save the MIDI file
notes_to_midi(mapped_notes, output_file='generated_music_fixed.mid')

# Download the MIDI file
from google.colab import files
files.download('generated_music_fixed.mid')


MIDI file saved as generated_music_fixed.mid


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>